# Cvičenie 6: Neurónové siete v PyTorch

Minulý týždeň ste už videli príklad implementácie neurónovej siete pomocou knižnice PyTorch. Na dnešnom cvičení sa pozrieme na to, ako presne funguje Autograd v PyTorch. Aby ste úspešne zvládli cvičenie, potrebujete mať nainštalovanú knižnicu PyTorch.

## 1. Výpočty v PyTorch

Na začiatok zadefinujeme jednoduchú operáciu sčítania pomocou PyTorch:

![](sources/lab06/6.1-addition.png)

In [ ]:
import torch

# create the nodes in the graph, and initialize values
a = torch.tensor(13)
b = torch.tensor(37)

# add together the two values
c = torch.add(a, b)
print(c)

Výstup výpočtu je tensor s hodnotou 50: `tensor(50)`. PyTorch podporuje tensory s hodnotami (môžu byť konštanty alebo premenné) a s výpočtami.

Na základe predošlého príkladu vytvorte viac zložitý graf:

![](sources/lab06/6.2-complicated-graph.png)

In [ ]:
# create the nodes in the graph, and initialize values
a = torch.tensor(2.5)
b = torch.tensor(6.5)

c = None
d = None
e = None

print(e)

## 2. Neurón v PyTorch

Neurón predstavuje základný výpočtový prvok neurónových sietí, ktorý pozostáva zo vstupov, synaptických váh, váženej sumy, aktivačnej funkcie a výstupnej funckie. V tomto kroku implementujeme jednoduchý neurón pomocou základných metód PyTorch, aby ste videli, ako tieto výpočty fungujú na úrovni s nižšou abstrakciou.

![](sources/lab06/6.3-perceptron.png)

In [ ]:
# simple perceptron with two input nodes
def my_neuron(x):
    # define some arbitrary weights for the two input values
    W = torch.tensor([[3, -2]], dtype=torch.float32)

    # define the bias of the neuron
    b = 1
    
    # compute weighted sum (hint: check out torch.matmul)
    z = None

    # apply the sigmoid activation function (hint: use torch.sigmoid)
    output = None

    return output

sample_input = torch.tensor([[-1, 2]], dtype=torch.float32)

result = my_neuron(sample_input)
print(result)

## 3. Plne prepojené vrstvy v PyTorch

Ak chceme vytvoriť neurónovú sieť, jednoduché neuróny musíme usporiadať do tzv. vrstiev. Oproti predošlému príkladu musíme tiež upraviť váhy tak, aby boli aktualizovateľné. V neurónových sieťach sa najčastejšie používajú plne prepojené vrstvy (*fully connected* alebo *linear*).

In [ ]:
# x: input values
# n_in: number of input nodes
# n_out: number of output nodes
def my_dense_layer(x, n_in, n_out):
    # define variable weights as a matrix and biases
    # initialize weights for one
    # initialize biases for zero
    W = torch.ones((n_in, n_out), requires_grad=True)
    b = None
    
    # compute weighted sum (hint: check out torch.matmul)
    z = None

    # apply the sigmoid activation function (hint: use torch.sigmoid)
    output = None

    return output

Ako aj pred tým, naše riešenie vieme otestovať zadaním ľubovoľných hodnôt (s dodržaním počtu vstupných a výstupných neurónov).

In [ ]:
sample_input = torch.tensor([[1, 2.]])
print(my_dense_layer(sample_input, n_in=2, n_out=3))

Podobne by sme postupovali pri definícii neurónovej siete s viacerými vrstvami, vidíme ale, že takýto zápis je zdĺhavý a neprehľadný. Práve preto bola vyvinutá knižnica `torch.nn`, ktorá obsahuje už naprogramované základné vrstvy pomocou jazyka C++.

## 4. Definícia a trénovanie neurónovej siete pomocou torch.nn

Pre dnešnú ukážku môžete používať ľubuvoľný dataset, odporúčame ale použiť niektorý dataset pre klasifikáciu z [tohto repozitára](https://archive.ics.uci.edu/ml/datasets.php).

Ako sme už videli minulý týždeň, v PyTorch potrebujeme vytvoriť sekvenčný model priamo s vrstvami, následne vytvoriť optimizátor a nakoniec natrénovať. Dnes ukážeme trocha iný prístup, kde najprv zadefinujete triedu pre model s vrstvami a prepojíte ich vo forward funkcii. Oba prístupy sú rovnocenné a môžete ich používať ľubovoľne. Najčastejšie sa používa vlastná trieda modelu, keďže je oveľa jednoduchšie manipulovať výstup vrstiev a vytvoriť viac vetiev.

In [11]:
from torch import nn

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(4, 10)
        self.relu = nn.ReLU(inplace=True)
        self.linear2 = nn.Linear(10, 3)
        self.sigmoid = nn.Sigmoid()
        
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.sigmoid(x)
        return x

model = Model()
print(model)

Model(
  (linear1): Linear(in_features=200, out_features=512, bias=True)
  (relu): ReLU(inplace=True)
  (linear2): Linear(in_features=512, out_features=1024, bias=True)
  (sigmoid): Sigmoid()
)


Užitočná metóda je metóda `print`, ktorá vám vypíše informácie o definovanej sieti.

Ak ste spokojní s topológiou siete, môžete vytvorit optimalizator a loss funkciu.

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

Ako Dataset si zvolíme Iris z minulých cvičené a použijeme už známy training loop

In [ ]:
import pandas as pd
dataset = pd.read_csv('iris.csv')
X = dataset.iloc[:, :4].values
y = dataset.iloc[:, -1].values

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
# transform string labels into number values 0, 1, 2
Y = encoder.fit_transform(y)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.Tensor(y_train)

In [ ]:
for epoch in range(100):
    optimizer.zero_grad()
    preds = model(X_train)
    loss = criterion(preds, y_train)
    loss.backward()
    optimizer.step()
    print("Loss:", loss.detach().item(), "accuracy:", (y_train== preds.argmax(-1)).sum().item()/len(y_train))

Ak chcete natrénovanú sieť používať, potrebujete zavolať model:

In [ ]:
y_pred = model(X_test)

**Poznámka**: ukážkové riešenie úloh s PyTorch nájdete [tu](sources/lab06/lab6-tensorflow-solution.py)).